In [2]:
#numpy loader method to load numpy arrays into the Pytorch ImageFolder dataset
import numpy as np

def my_numpy_loader(filename):
    return np.load(filename)

In [3]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, image):
        self.image = image
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((3, 0, 1, 2))
        #print(torch.from_numpy(image))
        return torch.from_numpy(image)

In [4]:
import os
os.chdir("../augmented_32x32x48")
cwd = os.getcwd()
print(cwd)

D:\github\augmented_32x32x48


In [5]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import visdom


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#Visdom Configuration
viz = visdom.Visdom()

# Hyper parameters
num_epochs = 100
num_classes = 2
batch_size = 100
learning_rate = 0.001


# Dicom dataset
train_dataset = torchvision.datasets.DatasetFolder(root='3D/train', 
                                           transform=ToTensor(), loader=my_numpy_loader, extensions="npy")

test_dataset = torchvision.datasets.DatasetFolder(root='3D/test', 
                                          transform=ToTensor(), loader=my_numpy_loader, extensions="npy")
print(train_dataset)
print(test_dataset)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=2):
        super(ConvNet,self).__init__()
        #Convo 1
        self.cnn1=nn.Conv3d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
        self.batch_norm1 = nn.BatchNorm3d(16)
        self.relu1=nn.ReLU()
        #Maxpool_1
        self.maxpool1=nn.MaxPool3d(kernel_size=2)
        #Convo_2
        self.cnn2=nn.Conv3d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
        self.batch_norm2 = nn.BatchNorm3d(32)
        self.relu2=nn.ReLU()
        #Maxpool_2
        self.maxpool2=nn.MaxPool3d(kernel_size=2)

        #Linear Layer
        self.fc1=nn.Linear(24576,1024)
        self.fc2=nn.Linear(1024,32)
        self.fc3=nn.Linear(32,2)

    def forward(self,x):
        #Convo_1
        out=self.cnn1(x)
        #print(out.shape)
        out=self.batch_norm1(out)
        #print(out.shape)
        out=self.relu1(out)
        #print(out.shape)
        
        #Max_pool1
        out=self.maxpool1(out)
        #print(out.shape)
        
        #Convo_2
        out=self.cnn2(out)
        #print(out.shape)
        out=self.batch_norm2(out)
        #print(out.shape)
        out=self.relu2(out)
        #print(out.shape)
        
        #Max_pool2
        out=self.maxpool2(out)
        #print(out.shape)
        out=out.view(out.size(0),-1)#Flattening out 
        #print(out.shape)
        
        #Linear Layers
        out=self.fc1(out)
        #print(out.shape)
        out=self.fc2(out)
        #print(out.shape)
        out=self.fc3(out)
        #print(out.shape)
        return out

model = ConvNet(num_classes).to(device).double()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
loss_window = viz.line(
    Y=torch.zeros((1)).cpu(),
    X=torch.zeros((1)).cpu(),
    opts=dict(xlabel='Epoch',ylabel='Loss',title='3D Training Loss'))

accuracy_window = viz.line(
    Y=torch.zeros((1)).cpu(),
    X=torch.zeros((1)).cpu(),
    opts=dict(xlabel='Epoch',ylabel='Accuracy [%]',title='3D Network Accuracy'))


counter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        loss_array = np.array([loss.item()])
        
        counter = counter + 1

    # Test the model
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Test Accuracy of the model on the 212 test images: {} %'.format(100 * correct / total))
        acc = 100 * correct / total
        
    viz.line(X=torch.ones((1,1)).cpu()*epoch, Y=torch.Tensor([loss]).unsqueeze(0),win=loss_window,update='append')
    viz.line(X=torch.ones((1,1)).cpu()*epoch, Y=torch.Tensor([acc]).unsqueeze(0),win=accuracy_window,update='append')

# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')

Dataset DatasetFolder
    Number of datapoints: 1964
    Root Location: 3D/train
    Transforms (if any): <__main__.ToTensor object at 0x00000263BF9E8908>
    Target Transforms (if any): None
Dataset DatasetFolder
    Number of datapoints: 844
    Root Location: 3D/test
    Transforms (if any): <__main__.ToTensor object at 0x00000263D8675278>
    Target Transforms (if any): None
Epoch [1/100], Step [1/20], Loss: 0.7101
Epoch [1/100], Step [2/20], Loss: 22.1492
Epoch [1/100], Step [3/20], Loss: 9.0828
Epoch [1/100], Step [4/20], Loss: 9.9354
Epoch [1/100], Step [5/20], Loss: 2.1996
Epoch [1/100], Step [6/20], Loss: 3.2673
Epoch [1/100], Step [7/20], Loss: 3.6926
Epoch [1/100], Step [8/20], Loss: 2.0021
Epoch [1/100], Step [9/20], Loss: 0.5904
Epoch [1/100], Step [10/20], Loss: 2.9334
Epoch [1/100], Step [11/20], Loss: 1.8872
Epoch [1/100], Step [12/20], Loss: 0.7116
Epoch [1/100], Step [13/20], Loss: 0.6593
Epoch [1/100], Step [14/20], Loss: 1.3466
Epoch [1/100], Step [15/20], Loss: 0.9

Epoch [9/100], Step [16/20], Loss: 0.3358
Epoch [9/100], Step [17/20], Loss: 0.4337
Epoch [9/100], Step [18/20], Loss: 0.2541
Epoch [9/100], Step [19/20], Loss: 0.3245
Epoch [9/100], Step [20/20], Loss: 0.2363
Test Accuracy of the model on the 212 test images: 85.30805687203791 %
Epoch [10/100], Step [1/20], Loss: 0.4564
Epoch [10/100], Step [2/20], Loss: 0.2770
Epoch [10/100], Step [3/20], Loss: 0.2463
Epoch [10/100], Step [4/20], Loss: 0.2642
Epoch [10/100], Step [5/20], Loss: 0.2904
Epoch [10/100], Step [6/20], Loss: 0.2790
Epoch [10/100], Step [7/20], Loss: 0.3356
Epoch [10/100], Step [8/20], Loss: 0.2881
Epoch [10/100], Step [9/20], Loss: 0.2519
Epoch [10/100], Step [10/20], Loss: 0.2934
Epoch [10/100], Step [11/20], Loss: 0.2469
Epoch [10/100], Step [12/20], Loss: 0.3887
Epoch [10/100], Step [13/20], Loss: 0.4063
Epoch [10/100], Step [14/20], Loss: 0.6026
Epoch [10/100], Step [15/20], Loss: 0.3311
Epoch [10/100], Step [16/20], Loss: 0.4128
Epoch [10/100], Step [17/20], Loss: 0.33

Epoch [18/100], Step [14/20], Loss: 0.1888
Epoch [18/100], Step [15/20], Loss: 0.3695
Epoch [18/100], Step [16/20], Loss: 0.1731
Epoch [18/100], Step [17/20], Loss: 0.1210
Epoch [18/100], Step [18/20], Loss: 0.1904
Epoch [18/100], Step [19/20], Loss: 0.2185
Epoch [18/100], Step [20/20], Loss: 0.0748
Test Accuracy of the model on the 212 test images: 93.36492890995261 %
Epoch [19/100], Step [1/20], Loss: 0.1016
Epoch [19/100], Step [2/20], Loss: 0.1784
Epoch [19/100], Step [3/20], Loss: 0.1985
Epoch [19/100], Step [4/20], Loss: 0.2217
Epoch [19/100], Step [5/20], Loss: 0.3800
Epoch [19/100], Step [6/20], Loss: 0.3457
Epoch [19/100], Step [7/20], Loss: 0.1179
Epoch [19/100], Step [8/20], Loss: 0.1857
Epoch [19/100], Step [9/20], Loss: 0.1245
Epoch [19/100], Step [10/20], Loss: 0.1621
Epoch [19/100], Step [11/20], Loss: 0.1425
Epoch [19/100], Step [12/20], Loss: 0.2166
Epoch [19/100], Step [13/20], Loss: 0.3191
Epoch [19/100], Step [14/20], Loss: 0.1774
Epoch [19/100], Step [15/20], Loss:

Epoch [27/100], Step [12/20], Loss: 0.1550
Epoch [27/100], Step [13/20], Loss: 0.0890
Epoch [27/100], Step [14/20], Loss: 0.1474
Epoch [27/100], Step [15/20], Loss: 0.1460
Epoch [27/100], Step [16/20], Loss: 0.0879
Epoch [27/100], Step [17/20], Loss: 0.2059
Epoch [27/100], Step [18/20], Loss: 0.1168
Epoch [27/100], Step [19/20], Loss: 0.0996
Epoch [27/100], Step [20/20], Loss: 0.1200
Test Accuracy of the model on the 212 test images: 96.09004739336493 %
Epoch [28/100], Step [1/20], Loss: 0.0525
Epoch [28/100], Step [2/20], Loss: 0.1079
Epoch [28/100], Step [3/20], Loss: 0.1366
Epoch [28/100], Step [4/20], Loss: 0.1059
Epoch [28/100], Step [5/20], Loss: 0.1260
Epoch [28/100], Step [6/20], Loss: 0.0888
Epoch [28/100], Step [7/20], Loss: 0.1367
Epoch [28/100], Step [8/20], Loss: 0.1152
Epoch [28/100], Step [9/20], Loss: 0.1068
Epoch [28/100], Step [10/20], Loss: 0.0730
Epoch [28/100], Step [11/20], Loss: 0.1619
Epoch [28/100], Step [12/20], Loss: 0.1588
Epoch [28/100], Step [13/20], Loss:

Epoch [36/100], Step [10/20], Loss: 0.1577
Epoch [36/100], Step [11/20], Loss: 0.1523
Epoch [36/100], Step [12/20], Loss: 0.2553
Epoch [36/100], Step [13/20], Loss: 0.1065
Epoch [36/100], Step [14/20], Loss: 0.0867
Epoch [36/100], Step [15/20], Loss: 0.1856
Epoch [36/100], Step [16/20], Loss: 0.1037
Epoch [36/100], Step [17/20], Loss: 0.0836
Epoch [36/100], Step [18/20], Loss: 0.0856
Epoch [36/100], Step [19/20], Loss: 0.1565
Epoch [36/100], Step [20/20], Loss: 0.1075
Test Accuracy of the model on the 212 test images: 95.14218009478672 %
Epoch [37/100], Step [1/20], Loss: 0.1080
Epoch [37/100], Step [2/20], Loss: 0.0426
Epoch [37/100], Step [3/20], Loss: 0.0917
Epoch [37/100], Step [4/20], Loss: 0.1224
Epoch [37/100], Step [5/20], Loss: 0.1471
Epoch [37/100], Step [6/20], Loss: 0.1046
Epoch [37/100], Step [7/20], Loss: 0.1315
Epoch [37/100], Step [8/20], Loss: 0.2109
Epoch [37/100], Step [9/20], Loss: 0.1219
Epoch [37/100], Step [10/20], Loss: 0.1505
Epoch [37/100], Step [11/20], Loss:

...

Epoch [63/100], Step [4/20], Loss: 0.0141
Epoch [63/100], Step [5/20], Loss: 0.0075
Epoch [63/100], Step [6/20], Loss: 0.0556
Epoch [63/100], Step [7/20], Loss: 0.0135
Epoch [63/100], Step [8/20], Loss: 0.0093
Epoch [63/100], Step [9/20], Loss: 0.0115
Epoch [63/100], Step [10/20], Loss: 0.0156
Epoch [63/100], Step [11/20], Loss: 0.0032
Epoch [63/100], Step [12/20], Loss: 0.0083
Epoch [63/100], Step [13/20], Loss: 0.0057
Epoch [63/100], Step [14/20], Loss: 0.0284
Epoch [63/100], Step [15/20], Loss: 0.0156
Epoch [63/100], Step [16/20], Loss: 0.0025
Epoch [63/100], Step [17/20], Loss: 0.0088
Epoch [63/100], Step [18/20], Loss: 0.0108
Epoch [63/100], Step [19/20], Loss: 0.0175
Epoch [63/100], Step [20/20], Loss: 0.0076
Test Accuracy of the model on the 212 test images: 98.69668246445498 %
Epoch [64/100], Step [1/20], Loss: 0.0027
Epoch [64/100], Step [2/20], Loss: 0.0099
Epoch [64/100], Step [3/20], Loss: 0.0243
Epoch [64/100], Step [4/20], Loss: 0.0055
Epoch [64/100], Step [5/20], Loss: 0

Epoch [72/100], Step [2/20], Loss: 0.0009
Epoch [72/100], Step [3/20], Loss: 0.0011
Epoch [72/100], Step [4/20], Loss: 0.0035
Epoch [72/100], Step [5/20], Loss: 0.0006
Epoch [72/100], Step [6/20], Loss: 0.0015
Epoch [72/100], Step [7/20], Loss: 0.0007
Epoch [72/100], Step [8/20], Loss: 0.0036
Epoch [72/100], Step [9/20], Loss: 0.0023
Epoch [72/100], Step [10/20], Loss: 0.0033
Epoch [72/100], Step [11/20], Loss: 0.0036
Epoch [72/100], Step [12/20], Loss: 0.0041
Epoch [72/100], Step [13/20], Loss: 0.0020
Epoch [72/100], Step [14/20], Loss: 0.0019
Epoch [72/100], Step [15/20], Loss: 0.0048
Epoch [72/100], Step [16/20], Loss: 0.0031
Epoch [72/100], Step [17/20], Loss: 0.0030
Epoch [72/100], Step [18/20], Loss: 0.0012
Epoch [72/100], Step [19/20], Loss: 0.0009
Epoch [72/100], Step [20/20], Loss: 0.0078
Test Accuracy of the model on the 212 test images: 98.45971563981043 %
Epoch [73/100], Step [1/20], Loss: 0.0050
Epoch [73/100], Step [2/20], Loss: 0.0033
Epoch [73/100], Step [3/20], Loss: 0

Epoch [81/100], Step [1/20], Loss: 0.0010
Epoch [81/100], Step [2/20], Loss: 0.0011
Epoch [81/100], Step [3/20], Loss: 0.0017
Epoch [81/100], Step [4/20], Loss: 0.0014
Epoch [81/100], Step [5/20], Loss: 0.0006
Epoch [81/100], Step [6/20], Loss: 0.0010
Epoch [81/100], Step [7/20], Loss: 0.0002
Epoch [81/100], Step [8/20], Loss: 0.0004
Epoch [81/100], Step [9/20], Loss: 0.0014
Epoch [81/100], Step [10/20], Loss: 0.0004
Epoch [81/100], Step [11/20], Loss: 0.0006
Epoch [81/100], Step [12/20], Loss: 0.0039
Epoch [81/100], Step [13/20], Loss: 0.0006
Epoch [81/100], Step [14/20], Loss: 0.0005
Epoch [81/100], Step [15/20], Loss: 0.0003
Epoch [81/100], Step [16/20], Loss: 0.0010
Epoch [81/100], Step [17/20], Loss: 0.0013
Epoch [81/100], Step [18/20], Loss: 0.0006
Epoch [81/100], Step [19/20], Loss: 0.0017
Epoch [81/100], Step [20/20], Loss: 0.0002
Test Accuracy of the model on the 212 test images: 98.81516587677726 %
Epoch [82/100], Step [1/20], Loss: 0.0003
Epoch [82/100], Step [2/20], Loss: 0

Test Accuracy of the model on the 212 test images: 98.69668246445498 %
Epoch [90/100], Step [1/20], Loss: 0.0002
Epoch [90/100], Step [2/20], Loss: 0.0003
Epoch [90/100], Step [3/20], Loss: 0.0002
Epoch [90/100], Step [4/20], Loss: 0.0003
Epoch [90/100], Step [5/20], Loss: 0.0004
Epoch [90/100], Step [6/20], Loss: 0.0005
Epoch [90/100], Step [7/20], Loss: 0.0002
Epoch [90/100], Step [8/20], Loss: 0.0002
Epoch [90/100], Step [9/20], Loss: 0.0002
Epoch [90/100], Step [10/20], Loss: 0.0005
Epoch [90/100], Step [11/20], Loss: 0.0007
Epoch [90/100], Step [12/20], Loss: 0.0017
Epoch [90/100], Step [13/20], Loss: 0.0002
Epoch [90/100], Step [14/20], Loss: 0.0002
Epoch [90/100], Step [15/20], Loss: 0.0006
Epoch [90/100], Step [16/20], Loss: 0.0004
Epoch [90/100], Step [17/20], Loss: 0.0004
Epoch [90/100], Step [18/20], Loss: 0.0010
Epoch [90/100], Step [19/20], Loss: 0.0004
Epoch [90/100], Step [20/20], Loss: 0.0005
Test Accuracy of the model on the 212 test images: 98.81516587677726 %
Epoch [

Epoch [98/100], Step [19/20], Loss: 0.0002
Epoch [98/100], Step [20/20], Loss: 0.0004
Test Accuracy of the model on the 212 test images: 98.69668246445498 %
Epoch [99/100], Step [1/20], Loss: 0.0001
Epoch [99/100], Step [2/20], Loss: 0.0002
Epoch [99/100], Step [3/20], Loss: 0.0004
Epoch [99/100], Step [4/20], Loss: 0.0003
Epoch [99/100], Step [5/20], Loss: 0.0001
Epoch [99/100], Step [6/20], Loss: 0.0004
Epoch [99/100], Step [7/20], Loss: 0.0003
Epoch [99/100], Step [8/20], Loss: 0.0002
Epoch [99/100], Step [9/20], Loss: 0.0004
Epoch [99/100], Step [10/20], Loss: 0.0002
Epoch [99/100], Step [11/20], Loss: 0.0002
Epoch [99/100], Step [12/20], Loss: 0.0001
Epoch [99/100], Step [13/20], Loss: 0.0002
Epoch [99/100], Step [14/20], Loss: 0.0005
Epoch [99/100], Step [15/20], Loss: 0.0002
Epoch [99/100], Step [16/20], Loss: 0.0006
Epoch [99/100], Step [17/20], Loss: 0.0001
Epoch [99/100], Step [18/20], Loss: 0.0001
Epoch [99/100], Step [19/20], Loss: 0.0001
Epoch [99/100], Step [20/20], Loss: